## streaming outpup Mode

1. complete

전체 결과를 다시 출력
전체 집합을 보고 싶을때 사용
필수 상황 : groupBy().count() 같은 단순 집계시 사용

2. update

새로운데이터 + 값이 바뀐 데이터만 출력

3. append

새로 추가된 데이터만 출력
계속 늘어나는 로그성 데이터 적합
주의 ! groupBy같은 집계는 못 사용한다 -> 과거 데이터를 알 수 없기때문


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder.appName("StructuredNetworkWordCount").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/03 04:35:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
lines=(
    spark
    .readStream
    .format("socket")
    .option("host","localhost")
    .option("port",9999)
    .load()
)
words=lines.select(
    F.explode(
        F.split(lines.value," ")
    ).alias("word")
)
wordCounts=words.groupBy("word").count()
query=(
    wordCounts
    .writeStream
    .outputMode("complete")
    .format("console")
    .start()
)
query.awaitTermination()

26/02/03 04:38:46 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.
26/02/03 04:38:47 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-d20a4edf-83aa-497b-9477-6e67dc80b29d. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
26/02/03 04:38:47 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
                                                                                

-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|Hello|    1|
+-----+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|Hello|    1|
|world|    1|
+-----+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|Smith|    1|
|Hello|    2|
|world|    1|
+-----+-----+



26/02/03 04:39:23 WARN TextSocketMicroBatchStream: Stream closed by localhost:9999
ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.8/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 